# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [129]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [130]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [131]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'fruits_db', 'local', 'met', 'travel_db', 'uk_food']


In [132]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [133]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [134]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [135]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': '1 The Droveway',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'FirstLight Bar & Café',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.64041272672,
 'FHRSID': 1290215,
 'LocalAuthorityBusinessID': 'PI/000083029',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DH',
 'RatingDate': '2020-11-09T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('672f1922eec71a6554529162'),
 'geocode': {'latitude': '51.150625', 'longitude': '1.378231'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1290215',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '

In [136]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [137]:
# Create a dictionary for the new restaurant data
new_restaurant  = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [138]:
# Insert the new restaurant into the collection
result = db.establishments.insert_one(new_restaurant)
print("New restaurant inserted with _id:", result.inserted_id)

New restaurant inserted with _id: 672f1926e1fd831b97c556ce


In [139]:
# Check that the new restaurant was inserted
restaurant = db.establishments.find_one({'BusinessName': 'Penang Flavours'})
pprint(restaurant)

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('672f1926e1fd831b97c556ce'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [140]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
result = db.establishments.find_one(
    {'BusinessType': 'Restaurant/Cafe/Canteen'},
    {'BusinessTypeID': 1}
)
pprint(result)

{'BusinessTypeID': 1, '_id': ObjectId('672f1922eec71a6554529162')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [141]:
# Update the new restaurant with the correct BusinessTypeID
correct_id = result["BusinessTypeID"]
result = db.establishments.update_one(
        {'BusinessName': 'Penang Flavours'},
        {'$set': {'BusinessTypeID': correct_id}}
    )

In [142]:
# Confirm that the new restaurant was updated
updated_restaurant = db.establishments.find_one(
    {'BusinessName': 'Penang Flavours'},
    {'BusinessTypeID': 1}
)
pprint(updated_restaurant)

{'BusinessTypeID': 1, '_id': ObjectId('672f1926e1fd831b97c556ce')}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [143]:
# Find how many documents have LocalAuthorityName as "Dover"
count = db.establishments.count_documents({'LocalAuthorityName': 'Dover'})
pprint(count)

994


In [144]:
# Delete all documents where LocalAuthorityName is "Dover"
result = db.establishments.delete_many({'LocalAuthorityName': 'Dover'})
print(result.deleted_count)


994


In [145]:
# Check if any remaining documents include Dover
remaining = db.establishments.count_documents({'LocalAuthorityName': 'Dover'})
print(remaining)

0


In [146]:
# Check that other documents remain with 'find_one'
pprint(db.establishments.find_one())

{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': '4',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('672f1922eec71a655452944a'),
 'geocode': {'latitude': '51.086058', 'longitude': '1.196408'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          '

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [147]:
# Change the data type from String to Decimal for longitude and latitude
result = db.establishments.update_many(
    {'geocode.latitude': {'$type': 'string'}, 'geocode.longitude': {'$type': 'string'}},  # Only update if they are strings
    [
        {'$set': {
            'geocode.latitude': {'$toDecimal': '$geocode.latitude'},
            'geocode.longitude': {'$toDecimal': '$geocode.longitude'}
        }}
    ]
)

In [148]:
print(result.modified_count)

38786


Use `update_many` to convert `RatingValue` to integer numbers.

In [149]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [150]:
# Change the data type from String to Integer for RatingValue
result = db.establishments.update_many(
    {'RatingValue': {'$type': 'string'}},  # Only update if RatingValue is a string
    [
        {'$set': {
            'RatingValue': {'$toInt': '$RatingValue'}
        }}
    ]
)
print(result.matched_count)

34694


In [151]:
# Check that the coordinates and rating value are now numbers
record = db.establishments.find_one(
    {},
    {'geocode.latitude': 1, 'geocode.longitude': 1, 'RatingValue': 1, '_id': 0}
)
pprint(record)
print("Latitude:", type(record.get('geocode', {}).get('latitude')))
print("Longitude:", type(record.get('geocode', {}).get('longitude')))
print("RatingValue:", type(record.get('RatingValue')))

{'RatingValue': 4,
 'geocode': {'latitude': Decimal128('51.086058'),
             'longitude': Decimal128('1.196408')}}
Latitude: <class 'bson.decimal128.Decimal128'>
Longitude: <class 'bson.decimal128.Decimal128'>
RatingValue: <class 'int'>
